# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")
from pprint import pprint



# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chilecito,-29.1650,-67.4954,20.53,64,1,5.06,AR,-10800
1,1,ushuaia,-54.8061,-68.3185,6.83,81,0,7.20,AR,-10800
2,2,adamstown,-25.0667,-130.1002,20.91,76,100,5.24,PN,-28800
3,3,grytviken,-54.2806,-36.5098,0.45,80,10,6.25,GS,-7200
4,4,edinburgh of the seven seas,-37.0676,-12.3108,12.56,70,97,6.71,SH,0


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:


# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = "Humidity",
    scale = .5

)


# Display the map plot
map_plot_1



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_good_weather = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Max Temp'] > 21) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_df_good_weather = city_data_df_good_weather.dropna(axis=1,how='all')

# Display sample data
city_data_df_good_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,'ibri,23.2179,56.4922,25.23,35,0,2.66,OM,14400
188,188,basoda,23.8794,78.2872,21.87,53,0,1.18,IN,19800
219,219,hirna,27.9512,75.0639,23.22,29,0,2.36,IN,19800
329,329,hafar al-batin,28.4328,45.9628,21.32,88,0,3.09,SA,10800
356,356,tazacorte,28.6429,-17.9326,21.38,56,0,3.09,ES,0


### Step 3: Create a new DataFrame called `hotel_df`.

In [57]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df_good_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()
#print(hotel_df.count())

City          14
Country       14
Lat           14
Lng           14
Humidity      14
Hotel Name    14
dtype: int64


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
# Set parameters to search for a hotel
radius = '10000'
params = {
    "filter" : "",
    "bias" : "",
    "categories" : "accommodation.hotel&accommodation.hut&accommodation.apartment&accommodation.chalet&accommodation.guest_house&accommodation.hostel&accommodation.motel"
}

count = 0


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = hotel_df.loc[index,'Lat']
    Lng = hotel_df.loc[index,'Lng']
    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = str(f'{Lat},{Lng},10000')
    params["bias"] = str(f'{Lat},{Lng}')

    # Set base URL
    base_url = str(f'https://api.geoapify.com/v2/places?categories={params['categories']}&filter=circle:{params['filter']}&bias=proximity:{params['bias']}&limit=1&apiKey={geoapify_key}')

    # Make and API request using the params dictionary
    name_address = requests.get(base_url)

    # Convert the API response to JSON format
    name_address = name_address.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index,'Hotel Name'] = name_address['features'][0]['properties']['address_line1']
        #pprint(name_address)
        count = count+1
        print(f'{count} hotels found so far ✅')
        
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"


    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
1 hotels found so far ✅
'ibri - nearest hotel: Pūri — Auce — Grīvaiši
basoda - nearest hotel: No hotel found
hirna - nearest hotel: No hotel found
hafar al-batin - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
dajal - nearest hotel: No hotel found
morondava - nearest hotel: No hotel found
adrar - nearest hotel: No hotel found
2 hotels found so far ✅
al khaburah - nearest hotel: Sabina
borazjan - nearest hotel: No hotel found
inekar - nearest hotel: No hotel found
ayorou - nearest hotel: No hotel found
champoton - nearest hotel: No hotel found
lao cai - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
50,'ibri,OM,23.2179,56.4922,35,Pūri — Auce — Grīvaiši
188,basoda,IN,23.8794,78.2872,53,No hotel found
219,hirna,IN,27.9512,75.0639,29,No hotel found
329,hafar al-batin,SA,28.4328,45.9628,88,No hotel found
356,tazacorte,ES,28.6429,-17.9326,56,No hotel found
360,dajal,PK,29.5563,70.3797,25,No hotel found
416,morondava,MG,-20.2951,44.2786,80,No hotel found
418,adrar,DZ,27.8734,-0.2875,18,No hotel found
421,al khaburah,OM,23.9709,57.0911,44,Sabina
477,borazjan,IR,29.2661,51.2121,42,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [56]:

# Configure the map plot
city_data_df_good_weather_and_hotels = hotel_df.loc[(hotel_df['Hotel Name'] != 'No hotel found') ]

# Configure the map
map_plot_2 = city_data_df_good_weather_and_hotels.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = 1000,
    scale = 1,
    hover_cols=["Country", "Hotel Name", "City"],
    title = "Map: cities with perfect weather and hotels nearby"
    )


# Display the map plot
map_plot_2


# Configure the map 2
map_plot_3 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = 500,
    scale = 1,
    hover_cols=["Country", "Hotel Name", "City"],
    title = "All cities with good weather"
    )

# Display the map plot
display(map_plot_2)
display(map_plot_3)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name,City)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name,City)